In [11]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


# Fake data creation

In [12]:
from numpy import integer


fake=Faker()

fake_data = defaultdict(list)

row_number=int(input("Please write the number you wish to see the row size, DON'T CRASH ME!"))



In [13]:
# Creating sender and receiver ids 
for i in range(row_number):
    fake_data['sender_id'].append(fake.bothify(text='####'))
#fake_data['receiver_id']=fake_data['sender_id']

sender_id=fake_data["sender_id"]#Changing 'sender_id' column to a list
#receiver_id=fake_data["receiver_id"]#Changin 'receiver_id' column to a list

new_sender_id=[] # yeni data frame '  e  eklemek içni yeni from_custormer listem 
manipulated_receiver_id=[]# yeni data frame '  e  eklemek içni yeni to_ custormer listem 
amount=[]
timestamp=[]

Enjecting fake_data to DataFrame

df=pd.DataFrame(fake_data)
df=df[['sender_id','receiver_id']] 
df.head()

In [14]:
for idx in range(len(fake_data['sender_id'])): # fake_data ['sender_id'] nin uzunluğu kadar bir döngü yapiyorum
    # rastgele random sayi üretiyorum ki from_customer ' random sayida aktivasyonu olsun
    #f_c_s=''.join(sender_id[i])
    temp_sender_val=sender_id[idx]
    temp_receiver_list=[]
    temp_amount_list=[]
    temp_timestamp_list=[]

    for j in range(random.randint(1,10)): #random sayiyla döngü kuruyorum 

        # ayni müşteri kendine göndermemesi iiçin seçimi parçaliyorum
        
        rnd_reciever_id=''.join(random.choices(sender_id))         #to_customer listesinden random seçiyorum 
        
        if (temp_sender_val== rnd_reciever_id):  # ayni müşteri kendine göndermemesi id leri  kiyasliyorum eğer ayniysa döngüye devam ediyor
            continue

        temp_receiver_list.append(rnd_reciever_id)
        temp_amount_list.append(fake.pyfloat(right_digits=2,positive=True,min_value=2.00,max_value=10000.00))
        temp_timestamp_list.append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
    
    if len(new_sender_id)==len(fake_data['sender_id']): # fake_data['sender_id'] nin uzunluğuna eşit olunca çikmasi lazim
            break
    new_sender_id.append(temp_sender_val) # ana listeden yeni listeye random sayisi kadar  aynisindan ekliyo ki gönderenle alan eşit olsun
    
        
    if len(manipulated_receiver_id)==len(fake_data['sender_id']):
            break
    # random seçtiğimiz ögeyi yeni to_custoemr listesine ekliyoruz
    manipulated_receiver_id.append(temp_receiver_list)
    amount.append(temp_amount_list)
    timestamp.append(temp_timestamp_list)
     # random seçtiğimiz ögeyi yeni to_custoemr listesine ekliyoruz
        

In [15]:
fake_data['receiver_id']=manipulated_receiver_id
fake_data['amount']=amount
fake_data['timestamp']=timestamp
df=pd.DataFrame(fake_data)
df=df[['sender_id','receiver_id','amount','timestamp']]
df

,sender_id,receiver_id,amount,timestamp
0,7577,"[1251, 0570]","[7306.21, 768.93]","[2022-10-20 00:05:48, 2022-10-29 01:55:29]"
1,8796,"[3264, 6436, 4087, 6436, 3264, 9796]","[5217.22, 9467.72, 8999.34, 6590.6, 6520.34, 4...","[2022-10-05 04:55:58, 2022-10-29 12:30:13, 202..."
2,1251,"[8796, 3016]","[5862.28, 5813.2]","[2022-10-05 15:04:35, 2022-10-03 09:39:03]"
3,9796,"[7663, 3016, 0570, 3264, 6436, 1572, 9942, 7663]","[6505.7, 4536.75, 9985.19, 7194.51, 7617.7, 58...","[2022-10-16 07:59:10, 2022-11-03 01:07:48, 202..."
4,5178,"[3625, 0243, 3016, 7577, 5051, 3264, 0733]","[959.3, 1570.17, 4426.0, 3917.98, 3728.63, 659...","[2022-10-21 09:57:46, 2022-10-08 21:43:57, 202..."
5,5657,"[1572, 9383]","[8493.46, 3010.8]","[2022-10-12 21:52:56, 2022-10-26 13:30:47]"
6,7663,[5051],[6496.72],[2022-10-20 08:53:00]
7,5051,"[5657, 0733, 1572, 3504]","[1932.88, 1699.14, 6421.4, 6729.44]","[2022-09-24 13:03:49, 2022-09-24 22:48:24, 202..."
8,1908,"[1572, 9796, 3625, 6436]","[1414.83, 1233.11, 8565.16, 8559.83]","[2022-10-02 18:55:03, 2022-09-21 17:30:28, 202..."
9,9942,"[5178, 5178, 7577]","[1731.94, 6394.4, 2329.65]","[2022-09-16 23:03:03, 2022-10-27 22:08:22, 202..."


# Neo4j

In [ ]:
from neo4j import GraphDatabase

In [ ]:
transaction_info=df.values.tolist()

In [ ]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")

def create_customer(tx, customer_id):
    tx.run("CREATE (c1:customer {name: $name, customer_id: $id})",name='customer', id=customer_id)

def create_transaction(tx,customer_id,other_customer_id,amount,timestamp):
    tx.run("MATCH (c1:customer),(c2:customer) "
    "Where c1.customer_id= $id and c2.customer_id= $otherid "
    "Create (c1)-[t:Transaction{amount: $amount, time: $timestamp}]->(c2)",
    id=customer_id,otherid=other_customer_id,amount=amount,timestamp=timestamp)


with driver.session() as session:

    session.execute_write(clear_db)

    for i in range(len(transaction_info)):
        session.write_transaction(create_customer, str(transaction_info[i][0]))

    
    for element in transaction_info:
        for block in range(1,len(element)):
            for j in range(len(element[block])):

                session.write_transaction(create_transaction,element[0],element[1][j],element[2][j],element[3][j])

driver.close()